# Movie Success EDA by Wesley Giles

## Import necessary modules and environment variables

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
load_dotenv()

True

## Import data